In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('modified_imdb_user_ratings.csv')

active_users = df[df['User'].isin(['User_2', 'User_6', 'User_44'])]
print("\nActive users (User 2, User 6, User 44):")
print(active_users)
ratings_data = df.drop(columns=['User'])
ratings_data_filled = ratings_data.fillna(0)
cosine_sim_matrix = cosine_similarity(ratings_data_filled)
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=df['User'], columns=df['User'])
top_20_percent_users = {}

for active_user in active_users['User']:
    similar_users = cosine_sim_df[active_user].sort_values(ascending=False)
    similar_users = similar_users.drop(active_user)
    top_20_percent_count = int(len(similar_users) * 0.2)
    top_20_percent_users[active_user] = similar_users.head(top_20_percent_count)
    print(f"\nTop 20% closest users for Active User: {active_user}")
    print(top_20_percent_users[active_user])

predicted_ratings = {}
threshold = 3

for active_user in active_users['User']:
    print(f"\nPredicting missing ratings for Active User: {active_user}")
    user_ratings = df.loc[df['User'] == active_user]
    predicted_ratings_for_user = {}

    for item in df.columns[1:]:
        if pd.notna(user_ratings[item].values[0]):
            continue
        weighted_ratings = 0
        similarity_sum = 0

        for similar_user, similarity in top_20_percent_users[active_user].items():
            similar_user_rating = df.loc[df['User'] == similar_user, item].values[0]
            if not np.isnan(similar_user_rating):
                weighted_ratings += similarity * similar_user_rating
                similarity_sum += np.abs(similarity)

        predicted_rating = weighted_ratings / similarity_sum if similarity_sum != 0 else 0
        predicted_ratings_for_user[item] = predicted_rating
        classification = "Like" if predicted_rating >= threshold else "Dislike"
        print(f"Item: {item} | Predicted Rating: {predicted_rating:.2f} | Classification: {classification}")

    predicted_ratings[active_user] = predicted_ratings_for_user




Active users (User 2, User 6, User 44):
       User  The Shawshank Redemption  The Godfather  The Dark Knight  \
1    User_2                       3.0            4.0              3.0   
5    User_6                       4.0            3.0              NaN   
43  User_44                       4.0            3.0              3.0   

    The Godfather Part II  12 Angry Men  \
1                     3.0           3.0   
5                     3.0           4.0   
43                    NaN           4.0   

    The Lord of the Rings: The Return of the King  Schindler&apos;s List  \
1                                             4.0                    NaN   
5                                             3.0                    NaN   
43                                            4.0                    NaN   

    Pulp Fiction  The Lord of the Rings: The Fellowship of the Ring  \
1            2.0                                                NaN   
5            2.0                              

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


df = pd.read_csv('imdb_user_ratings.csv')
df.set_index('User', inplace=True)
tu = df.shape[0]
print(f"Total number of users (tu): {tu}")
tni = df.shape[1]
print(f"Total number of columns (tni): {tni}")
value_mapping = {
    7: 2,
    8: 3,
    9: 4,
    10: 5
}


df = df.applymap(lambda x: value_mapping.get(x, x) if pd.notna(x) and isinstance(x, (int, float)) else x)
ratings_matrix = df.fillna(0)
user_means = ratings_matrix.mean(axis=1)
mean_centered_matrix = ratings_matrix.subtract(user_means, axis=0)
cosine_sim_matrix = cosine_similarity(mean_centered_matrix)
active_users = ['User_2', 'User_6', 'User_44']
user_indices = {user: idx for idx, user in enumerate(ratings_matrix.index)}
beta = 0.5
discounted_similarities = {}

for active_user in active_users:
    active_user_index = user_indices[active_user]
    user_similarities = cosine_sim_matrix[active_user_index]
    similarity_scores = [(idx, sim) for idx, sim in enumerate(user_similarities) if idx != active_user_index]
    sorted_similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    discounted_similarities[active_user] = []
    for rank, (idx, sim) in enumerate(sorted_similarity_scores):
        df_value = 1 / (1 + beta * (rank + 1))
        ds = sim * df_value
        user_name = ratings_matrix.index[idx]
        discounted_similarities[active_user].append((user_name, df_value, ds))


    num_top_users = int(len(discounted_similarities[active_user]) * 0.2)
    top_20_percent_users = sorted(discounted_similarities[active_user], key=lambda x: x[2], reverse=True)[:num_top_users]
    discounted_similarities[active_user] = top_20_percent_users


predicted_ratings = {}

for active_user in active_users:

    top_users = discounted_similarities[active_user]
    user_ratings = ratings_matrix.loc[active_user].values
    unseen_items = np.where(user_ratings == 0)[0]
    predicted_ratings[active_user] = {}

    for item_idx in unseen_items:

        weighted_sum = 0
        total_weight = 0
        for user_name, _, ds in top_users:
            user_idx = ratings_matrix.index.get_loc(user_name)
            if ratings_matrix.iloc[user_idx, item_idx] != 0:
                rating = ratings_matrix.iloc[user_idx, item_idx]
                weighted_sum += ds * rating
                total_weight += ds

        if total_weight > 0:
            predicted_rating = weighted_sum / total_weight
        else:
            predicted_rating = user_means[active_user]
        predicted_ratings[active_user][item_idx] = predicted_rating
like_threshold = 3
for active_user, item_predictions in predicted_ratings.items():
    print(f"\nPredictions for {active_user}:")
    for item_idx, predicted_rating in item_predictions.items():
        if predicted_rating >= like_threshold:
            print(f"  Item {item_idx}: Predicted Rating = {predicted_rating:.2f} -> Like")
        else:
            print(f"  Item {item_idx}: Predicted Rating = {predicted_rating:.2f} -> Dislike")

Total number of users (tu): 50
Total number of columns (tni): 10

Predictions for User_2:
  Item 6: Predicted Rating = 3.00 -> Like
  Item 8: Predicted Rating = 2.00 -> Dislike
  Item 9: Predicted Rating = 2.11 -> Dislike

Predictions for User_6:
  Item 2: Predicted Rating = 3.16 -> Like
  Item 6: Predicted Rating = 2.40 -> Dislike

Predictions for User_44:
  Item 3: Predicted Rating = 3.06 -> Like
  Item 6: Predicted Rating = 3.11 -> Like
  Item 7: Predicted Rating = 2.23 -> Dislike
  Item 8: Predicted Rating = 2.20 -> Dislike
  Item 9: Predicted Rating = 2.34 -> Dislike


<ipython-input-4-20c9729573a9>:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: value_mapping.get(x, x) if pd.notna(x) and isinstance(x, (int, float)) else x)


In [5]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr


df = pd.read_csv('imdb_user_ratings.csv')
df.set_index('User', inplace=True)
tu = df.shape[0]
print(f"Total number of users (tu): {tu}")
tni = df.shape[1]
print(f"Total number of columns (tni): {tni}")


value_mapping = {
    7: 2,
    8: 3,
    9: 4,
    10: 5
}


df = df.applymap(lambda x: value_mapping.get(x, x) if pd.notna(x) and isinstance(x, (int, float)) else x)
ratings_matrix = df.fillna(0)
user_means = ratings_matrix.mean(axis=1)
mean_centered_matrix = ratings_matrix.subtract(user_means, axis=0)
def compute_pcc(user1, user2, ratings_matrix):

    common_ratings = ratings_matrix.loc[user1][ratings_matrix.loc[user1] > 0].index.intersection(
        ratings_matrix.loc[user2][ratings_matrix.loc[user2] > 0].index)

    if len(common_ratings) == 0:
        return 0

    ratings1 = ratings_matrix.loc[user1, common_ratings]
    ratings2 = ratings_matrix.loc[user2, common_ratings]
    return pearsonr(ratings1, ratings2)[0]


pcc_matrix = pd.DataFrame(index=ratings_matrix.index, columns=ratings_matrix.index)


for user1 in ratings_matrix.index:
    for user2 in ratings_matrix.index:
        pcc_matrix.loc[user1, user2] = compute_pcc(user1, user2, ratings_matrix)


active_users = ['User_2', 'User_6', 'User_44']


top_20_percent_users = {}

for active_user in active_users:
    similarity_scores = pcc_matrix.loc[active_user]
    similarity_scores = similarity_scores.drop(active_user)
    sorted_similarity_scores = similarity_scores.sort_values(ascending=False)
    num_top_users = int(len(sorted_similarity_scores) * 0.2)
    top_20_percent_users[active_user] = sorted_similarity_scores.head(num_top_users)


threshold_B = 0.5
discount_factor = 0.5
discounted_similarity = {}

for active_user in active_users:
    discounted_similarity[active_user] = {}
    top_users = top_20_percent_users[active_user]
    for similar_user, similarity in top_users.items():
        if similarity < threshold_B:

            df = discount_factor
            ds = similarity * df
            discounted_similarity[active_user][similar_user] = ds
        else:

            df = 1
            ds = similarity
            discounted_similarity[active_user][similar_user] = ds


        print(f"Active User: {active_user}, Similar User: {similar_user}, DF: {df}, DS: {ds:.4f}")

predictions = {}

for active_user in active_users:
    predictions[active_user] = {}


    user_ratings = ratings_matrix.loc[active_user]

    unrated_items = user_ratings[user_ratings == 0].index

    for item in unrated_items:
        weighted_sum = 0
        total_weight = 0

        for similar_user, similarity in discounted_similarity[active_user].items():
            if ratings_matrix.loc[similar_user, item] > 0:
                weighted_sum += similarity * ratings_matrix.loc[similar_user, item]
                total_weight += similarity


        if total_weight > 0:
            predicted_rating = weighted_sum / total_weight
        else:
            predicted_rating = 0

        predictions[active_user][item] = predicted_rating


print("\nPredictions for Active Users:")
for active_user, user_predictions in predictions.items():
    print(f"\n{active_user}:")
    for item, prediction in user_predictions.items():
        print(f"  Predicted rating for item {item}: {prediction:.4f}")

like_threshold = 3
for active_user, user_predictions in predictions.items():
    print(f"\n{active_user}'s predicted preferences:")
    for item, prediction in user_predictions.items():
        if prediction >= like_threshold:
            print(f"  Item {item}: Like")
        else:
            print(f"  Item {item}: Dislike")

Total number of users (tu): 50
Total number of columns (tni): 10


<ipython-input-5-95d8cf94a04a>:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: value_mapping.get(x, x) if pd.notna(x) and isinstance(x, (int, float)) else x)
<ipython-input-5-95d8cf94a04a>:42: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return pearsonr(ratings1, ratings2)[0]


Active User: User_2, Similar User: User_30, DF: 1, DS: 1.0000
Active User: User_2, Similar User: User_37, DF: 1, DS: 0.8686
Active User: User_2, Similar User: User_42, DF: 1, DS: 0.8677
Active User: User_2, Similar User: User_15, DF: 1, DS: 0.8672
Active User: User_2, Similar User: User_47, DF: 1, DS: 0.8402
Active User: User_2, Similar User: User_43, DF: 1, DS: 0.7746
Active User: User_2, Similar User: User_36, DF: 1, DS: 0.7559
Active User: User_2, Similar User: User_1, DF: 1, DS: 0.7455
Active User: User_2, Similar User: User_45, DF: 1, DS: 0.7303
Active User: User_6, Similar User: User_5, DF: 1, DS: 0.8874
Active User: User_6, Similar User: User_36, DF: 1, DS: 0.8677
Active User: User_6, Similar User: User_23, DF: 1, DS: 0.8672
Active User: User_6, Similar User: User_40, DF: 1, DS: 0.8660
Active User: User_6, Similar User: User_28, DF: 1, DS: 0.8165
Active User: User_6, Similar User: User_39, DF: 1, DS: 0.8165
Active User: User_6, Similar User: User_22, DF: 1, DS: 0.7620
Active Use